In [0]:
dbutils.widgets.text("start_date","2024-01-01")
dbutils.widgets.text("end_date","2025-12-01")

In [0]:
start_date=dbutils.widgets.get("start_date")
end_date=dbutils.widgets.get("end_date")

In [0]:
from pyspark.sql.functions import col,date_format,year,concat,lit

df=(spark.sql(f"""
             SELECT explode(
                 sequence(to_date('{start_date}'),to_date('{end_date}') ,interval 1 month )
             ) AS month_start_date
             """)
    )
#display(df)

df=df.withColumns({
    "date_key":date_format("month_start_date","yyyyMMdd").cast("int"),
    "year":year("month_start_date"),
    "month_name":date_format("month_start_date","MMMM"),
    "short_month_name":date_format("month_start_date","MMM"),
    "quarter":concat(lit("Q"),date_format("month_start_date","Q")),
    "year_quarter":concat("year","quarter")
})

display(df)

df.write.mode("overwrite").format("delta").saveAsTable("fmcg.gold.dim_date")